# Causal Inference: Potential Outcomes and Randomization

**Based on Green (2022), *A Primer for Experimental Research on the Effectiveness of Campaign Communication*, Chapters 1-2**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Persuasion-at-Scale/causal-inference-intro/blob/main/green-causal-inference.ipynb)

This notebook walks through four core ideas:
1. **Potential outcomes** - the framework for defining causal effects
2. **Randomization** - why it produces unbiased estimates
3. **Selection bias** - what goes wrong without randomization
4. **Experiment analysis** - putting it all together

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from scipy import stats

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

BLUE = '#0072B2'
ORANGE = '#D55E00'
GREEN = '#009E73'
np.random.seed(42)

---
## 1. Potential Outcomes (Green, Table 2.1)

Every subject has **two** potential outcomes:
- $Y_i(1)$ = outcome if subject $i$ receives treatment
- $Y_i(0)$ = outcome if subject $i$ receives control

The **individual causal effect** is $\tau_i = Y_i(1) - Y_i(0)$.

The **Average Treatment Effect (ATE)** is the mean of all individual effects:
$$\text{ATE} = \frac{1}{N} \sum_{i=1}^{N} \left[ Y_i(1) - Y_i(0) \right]$$

Green's example: a cash subsidy program that might affect BMI. Five subjects, and (by divine omniscience) we know both potential outcomes for each.

In [ ]:
# Green (2022) Table 2.1 - God's-eye view
table21 = pd.DataFrame({
    'Subject': [1, 2, 3, 4, 5],
    'Y(1): BMI if treated': [16, 20, 25, 27, 22],
    'Y(0): BMI if control': [11, 12, 25, 29, 18],
    'Effect: Y(1)-Y(0)': [5, 8, 0, -2, 4],
})
table21 = table21.set_index('Subject')

print("Table 2.1: Potential outcomes for 5 subjects")
print("="*55)
display(table21)

true_ate = table21['Effect: Y(1)-Y(0)'].mean()
print(f"\nTrue ATE = {true_ate:.1f} BMI points")

### The Fundamental Problem of Causal Inference

In reality, we **never** see both columns. Each subject is either treated or not. We observe one potential outcome and the other is forever missing - the "counterfactual."

Suppose subjects 1 and 2 happen to be treated, and 3, 4, 5 are controls. What does the researcher actually see?

In [ ]:
# What the researcher actually observes
Y1 = np.array([16, 20, 25, 27, 22])  # potential outcome if treated
Y0 = np.array([11, 12, 25, 29, 18])  # potential outcome if control
d  = np.array([1, 1, 0, 0, 0])       # treatment assignment

Y_obs = d * Y1 + (1 - d) * Y0  # observed outcomes

observed = pd.DataFrame({
    'Subject': [1, 2, 3, 4, 5],
    'Treated?': ['Yes', 'Yes', 'No', 'No', 'No'],
    'Observed BMI': Y_obs.astype(int),
    'Y(1)': ['16', '20', '?', '?', '?'],
    'Y(0)': ['?', '?', '25', '29', '18'],
}).set_index('Subject')

display(observed)

naive_treated = Y_obs[d == 1].mean()
naive_control = Y_obs[d == 0].mean()
naive_estimate = naive_treated - naive_control

print(f"Mean BMI (treated):  {naive_treated:.1f}")
print(f"Mean BMI (control):  {naive_control:.1f}")
print(f"Naive estimate:      {naive_estimate:.1f}")
print(f"True ATE:            {true_ate:.1f}")
print(f"\nThe naive estimate is OFF by {naive_estimate - true_ate:.1f} points!")
print("This is SELECTION BIAS: the treated subjects were different to begin with.")

---
## 2. Randomization Fixes Everything

With 5 subjects and 2 assigned to treatment, there are $\binom{5}{2} = 10$ possible random assignments.

Green's key claim: the **average** of the difference-in-means across all possible randomizations equals the true ATE. Randomization doesn't guarantee a correct answer for any single experiment, but it's **unbiased** - correct on average.

In [ ]:
# Enumerate all 10 possible random assignments
subjects = list(range(5))
all_assignments = list(combinations(subjects, 2))  # choose 2 for treatment

results = []
for treated_idx in all_assignments:
    d_iter = np.zeros(5, dtype=int)
    d_iter[list(treated_idx)] = 1
    Y_obs_iter = d_iter * Y1 + (1 - d_iter) * Y0
    est = Y_obs_iter[d_iter == 1].mean() - Y_obs_iter[d_iter == 0].mean()
    results.append({
        'Treated subjects': f"{treated_idx[0]+1}, {treated_idx[1]+1}",
        'Mean(treated)': Y_obs_iter[d_iter == 1].mean(),
        'Mean(control)': Y_obs_iter[d_iter == 0].mean(),
        'Estimate': est
    })

results_df = pd.DataFrame(results)
display(results_df.round(2))

avg_estimate = results_df['Estimate'].mean()
print(f"\nAverage of all 10 estimates: {avg_estimate:.1f}")
print(f"True ATE:                    {true_ate:.1f}")
print(f"Match? {np.isclose(avg_estimate, true_ate)}  (unbiasedness!)")

In [ ]:
# Visualize the distribution of estimates
fig, ax = plt.subplots(figsize=(10, 5))

estimates = results_df['Estimate'].values
colors = [ORANGE if e < 0 else BLUE for e in estimates]

ax.scatter(estimates, np.ones(len(estimates)), c=colors, s=200, zorder=3, edgecolors='white', linewidth=1.5)

# Jitter overlapping points
from collections import Counter
counts = Counter(estimates)
for val, count in counts.items():
    if count > 1:
        positions = np.linspace(0.92, 1.08, count)
        idxs = [i for i, e in enumerate(estimates) if e == val]
        for idx, pos in zip(idxs, positions):
            ax.scatter([val], [pos], c=colors[idx], s=200, zorder=3, edgecolors='white', linewidth=1.5)

ax.axvline(true_ate, color=GREEN, linewidth=3, label=f'True ATE = {true_ate:.0f}', linestyle='-')
ax.axvline(avg_estimate, color='black', linewidth=2, label=f'Mean of estimates = {avg_estimate:.0f}', linestyle='--')

ax.set_xlabel('Difference-in-means estimate')
ax.set_yticks([])
ax.set_title('All 10 possible randomizations (2 treated out of 5 subjects)')
ax.legend(fontsize=12)

# Label negative estimates
ax.annotate('Negative estimates exist!\n(bad luck of the draw)', xy=(-7, 1.05),
            fontsize=10, color=ORANGE, ha='center')

plt.tight_layout()
plt.show()

---
## 3. Selection Bias: The SAT Prep Example

Green Ch 1 uses a running example: do SAT prep courses improve scores?

**The problem:** Students who sign up for SAT prep are systematically different from those who don't. They tend to be more motivated, have more resources, and were already better students. So comparing course-takers to non-takers confounds the course effect with pre-existing differences.

No matter how many covariates you control for, there's always another unmeasured confounder lurking. Green calls this the "skeptic's challenge."

Let's simulate this.

In [ ]:
# Simulate SAT prep self-selection
N = 500
TRUE_EFFECT = 30  # true effect of SAT prep course (points)

# Latent traits
ability = np.random.normal(500, 80, N)      # baseline ability
motivation = np.random.normal(0, 1, N)       # motivation (unobserved)

# Self-selection: higher ability AND motivation -> more likely to take the course
propensity = 0.005 * (ability - 500) + 0.8 * motivation
p_treat = 1 / (1 + np.exp(-propensity))
took_course = np.random.binomial(1, p_treat)

# Outcomes: score depends on ability, motivation, AND the course
noise = np.random.normal(0, 30, N)
score = ability + 20 * motivation + TRUE_EFFECT * took_course + noise

sat = pd.DataFrame({
    'ability': ability,
    'motivation': motivation,
    'took_course': took_course,
    'score': score
})

naive_diff = sat.groupby('took_course')['score'].mean()
naive_est = naive_diff[1] - naive_diff[0]

print(f"True course effect:  {TRUE_EFFECT} points")
print(f"Naive estimate:      {naive_est:.1f} points")
print(f"Bias:                {naive_est - TRUE_EFFECT:.1f} points (selection bias!)")
print(f"\nStudents in course:  {took_course.sum()} / {N}")

In [ ]:
# Visualize the selection bias
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Ability distributions by group
ax = axes[0]
ax.hist(sat.loc[sat['took_course']==0, 'ability'], bins=30, alpha=0.6,
        color=ORANGE, label='No course', density=True)
ax.hist(sat.loc[sat['took_course']==1, 'ability'], bins=30, alpha=0.6,
        color=BLUE, label='Took course', density=True)
ax.set_xlabel('Baseline ability')
ax.set_ylabel('Density')
ax.set_title('Self-selection: course-takers have higher ability')
ax.legend()

# Panel 2: Outcomes by group
ax = axes[1]
groups = sat.groupby('took_course')['score']
means = groups.mean()
sems = groups.sem()
x = [0, 1]
bars = ax.bar(x, means, yerr=1.96*sems, capsize=8,
              color=[ORANGE, BLUE], width=0.5)
ax.set_xticks(x)
ax.set_xticklabels(['No course', 'Took course'])
ax.set_ylabel('SAT score')
ax.set_title('Outcomes: naive comparison is misleading')

# Annotate
mid_y = means.mean()
ax.annotate(f'Naive diff = {naive_est:.0f}',
            xy=(0.5, mid_y), fontsize=13, ha='center', color='red', fontweight='bold')
ax.annotate(f'True effect = {TRUE_EFFECT}',
            xy=(0.5, mid_y - 30), fontsize=13, ha='center', color=GREEN, fontweight='bold')

plt.suptitle('Selection Bias in the SAT Prep Example', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Now: randomize treatment on the SAME population
random_assignment = np.random.binomial(1, 0.5, N)
score_rct = ability + 20 * motivation + TRUE_EFFECT * random_assignment + noise

rct = pd.DataFrame({
    'ability': ability,
    'assigned_course': random_assignment,
    'score': score_rct
})

rct_diff = rct.groupby('assigned_course')['score'].mean()
rct_est = rct_diff[1] - rct_diff[0]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Ability distributions (now balanced!)
ax = axes[0]
ax.hist(rct.loc[rct['assigned_course']==0, 'ability'], bins=30, alpha=0.6,
        color=ORANGE, label='Control', density=True)
ax.hist(rct.loc[rct['assigned_course']==1, 'ability'], bins=30, alpha=0.6,
        color=BLUE, label='Treatment', density=True)
ax.set_xlabel('Baseline ability')
ax.set_ylabel('Density')
ax.set_title('Random assignment: groups are balanced')
ax.legend()

# Panel 2: Outcomes
ax = axes[1]
means_rct = rct.groupby('assigned_course')['score'].mean()
sems_rct = rct.groupby('assigned_course')['score'].sem()
bars = ax.bar(x, means_rct, yerr=1.96*sems_rct, capsize=8,
              color=[ORANGE, BLUE], width=0.5)
ax.set_xticks(x)
ax.set_xticklabels(['Control', 'Treatment'])
ax.set_ylabel('SAT score')
ax.set_title('Experiment: estimate is close to truth')

mid_y_rct = means_rct.mean()
ax.annotate(f'RCT estimate = {rct_est:.0f}',
            xy=(0.5, mid_y_rct), fontsize=13, ha='center', color=BLUE, fontweight='bold')
ax.annotate(f'True effect = {TRUE_EFFECT}',
            xy=(0.5, mid_y_rct - 30), fontsize=13, ha='center', color=GREEN, fontweight='bold')

plt.suptitle('Randomization Eliminates Selection Bias', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"RCT estimate:    {rct_est:.1f} points")
print(f"True effect:     {TRUE_EFFECT} points")
print(f"Bias:            {rct_est - TRUE_EFFECT:.1f} points (sampling noise, not systematic bias)")

---
## 4. Running a Simple Experiment

Let's simulate a complete experiment and analyze it. We'll:
1. Generate a population with known potential outcomes
2. Randomly assign treatment
3. Compute the difference-in-means, standard error, confidence interval, and p-value
4. Repeat 1,000 times to see the sampling distribution

In [ ]:
# One experiment
N_exp = 200
ATE_true = 5.0

# Generate potential outcomes
Y0_pop = np.random.normal(50, 10, N_exp)
Y1_pop = Y0_pop + ATE_true + np.random.normal(0, 3, N_exp)  # heterogeneous effects

# Random assignment
treated = np.random.binomial(1, 0.5, N_exp)
Y_observed = treated * Y1_pop + (1 - treated) * Y0_pop

# Analysis
y_t = Y_observed[treated == 1]
y_c = Y_observed[treated == 0]

diff_means = y_t.mean() - y_c.mean()
se = np.sqrt(y_t.var(ddof=1)/len(y_t) + y_c.var(ddof=1)/len(y_c))
ci_low = diff_means - 1.96 * se
ci_high = diff_means + 1.96 * se
t_stat, p_val = stats.ttest_ind(y_t, y_c)

print("Single Experiment Results")
print("=" * 40)
print(f"N treated:     {treated.sum()}")
print(f"N control:     {N_exp - treated.sum()}")
print(f"Estimate:      {diff_means:.2f}")
print(f"Std error:     {se:.2f}")
print(f"95% CI:        [{ci_low:.2f}, {ci_high:.2f}]")
print(f"p-value:       {p_val:.4f}")
print(f"True ATE:      {ATE_true:.1f}")
print(f"CI covers truth? {ci_low <= ATE_true <= ci_high}")

In [ ]:
# Repeat 1000 times
n_sims = 1000
estimates = np.zeros(n_sims)
ci_lows = np.zeros(n_sims)
ci_highs = np.zeros(n_sims)

for sim in range(n_sims):
    Y0_sim = np.random.normal(50, 10, N_exp)
    Y1_sim = Y0_sim + ATE_true + np.random.normal(0, 3, N_exp)
    d_sim = np.random.binomial(1, 0.5, N_exp)
    Y_sim = d_sim * Y1_sim + (1 - d_sim) * Y0_sim

    yt = Y_sim[d_sim == 1]
    yc = Y_sim[d_sim == 0]
    est = yt.mean() - yc.mean()
    se_sim = np.sqrt(yt.var(ddof=1)/len(yt) + yc.var(ddof=1)/len(yc))

    estimates[sim] = est
    ci_lows[sim] = est - 1.96 * se_sim
    ci_highs[sim] = est + 1.96 * se_sim

coverage = np.mean((ci_lows <= ATE_true) & (ATE_true <= ci_highs))

# Plot sampling distribution
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(estimates, bins=40, color=BLUE, alpha=0.7, edgecolor='white', density=True)
ax.axvline(ATE_true, color=GREEN, linewidth=3, label=f'True ATE = {ATE_true}')
ax.axvline(estimates.mean(), color='black', linewidth=2, linestyle='--',
           label=f'Mean of estimates = {estimates.mean():.2f}')

ax.set_xlabel('Difference-in-means estimate')
ax.set_ylabel('Density')
ax.set_title(f'Sampling distribution: 1,000 experiments (N={N_exp} each)')
ax.legend(fontsize=12)

plt.tight_layout()
plt.show()

print(f"Mean of 1,000 estimates: {estimates.mean():.2f}  (true ATE = {ATE_true})")
print(f"SD of estimates:         {estimates.std():.2f}")
print(f"95% CI coverage:         {coverage:.1%}  (should be ~95%)")

---
## Summary: Three Assumptions for Unbiased Estimation

Green (Ch 2) identifies three core assumptions that, if met, make the difference-in-means an unbiased estimator of the ATE:

**1. Random assignment**
Treatment is independent of potential outcomes. Each subject had the same probability of being treated, determined by chance rather than choice. Threats: tampering, differential attrition.

**2. Noninterference (SUTVA)**
My outcomes depend only on MY treatment status, not on whether others are treated. Threats: spillover effects, competition for limited resources, social interaction between treatment and control groups.

**3. Symmetry**
Treatment is the ONLY systematic difference between groups. The treated group didn't also get extra attention, different measurement procedures, or a compound bundle of changes. Threats: Hawthorne effects, compound treatments, differential measurement.